In [1]:
import numpy as np
import readline
%pylab inline
%load_ext autoreload
%autoreload 2
%precision 3

Populating the interactive namespace from numpy and matplotlib



Bad key "text.kerning_factor" on line 4 in
/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


'%.3f'

In [4]:
# from gensim.test.utils import common_texts
# from gensim.corpora.dictionary import Dictionary
# # Create a corpus from a list of texts
# common_dictionary = Dictionary(common_texts)
# dict(common_dictionary)

In [5]:
# common_texts

In [6]:
# from gensim.models import LdaModel
# from collections import Counter
# common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]
# # common_corpus = [list(Counter(text).items()) for text in common_texts]
# # Train the model on the corpus.
# lda = LdaModel(common_corpus, id2word = common_dictionary, num_topics=3)

In [7]:
# lda.get_topics()

In [9]:
# print(lda.print_topic(0, 5))
# print(lda.print_topic(1, 5))
# # print(lda.print_topic(2, 5))

### tokenisers

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer
lemma = nltk.wordnet.WordNetLemmatizer()

tt = TweetTokenizer(preserve_case=False)

from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
stops.update({"#semst", "...", "rt"})
ps = nltk.stem.porter.PorterStemmer()

def tokenize(t):
    tokens = tt.tokenize(t)
    tokens = [tok for tok in tokens if not ((len(tok) == 1 and not tok.isalpha()) or tok in stops)]
    tokens = [ (ps.stem(tok) if tok[0] != "#" else tok) for tok in tokens]
    return tokens

### train and test data

In [ ]:
train = pd.read_csv("../semeval2016.6/StanceDataset/train.csv", encoding= 'unicode_escape', engine='python')
del train["Opinion Towards"]
del train["Sentiment"]
train["tokens"] = [ tokenize(t) for t in train["Tweet"].values ]
train["num_tokens"] = [len(tt) for tt in train["tokens"].values ]
train.head()

In [ ]:
test = pd.read_csv("../semeval2016.6/StanceDataset/test.csv", encoding= 'unicode_escape', engine='python')
del test["Opinion Towards"]
del test["Sentiment"]
test["tokens"] = [ tokenize(t) for t in test["Tweet"].values ]
test["num_tokens"] = [len(tt) for tt in test["tokens"].values ]
test.head()

In [ ]:
plt.hist(train['num_tokens'])

In [ ]:
# train["tokens"].values

In [ ]:
from collections import Counter
from itertools import chain, product
all_tokens = Counter(chain(*train["tokens"]))
plt.hist(np.log2(list(all_tokens.values()) ), log=True)
plt.show()

In [ ]:
all_tokens.most_common(10)

In [ ]:
all_tokens.most_common()[-20:]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer as BoW1
bow = BoW1(tokenizer = lambda x: x, lowercase = False, min_df = 2)
X = bow.fit_transform(train["tokens"].values)
X.shape

In [ ]:
# topic_idxs = {}
# for topic in set(train["Target"]):
#     train_idxs = np.where(train["Target"] == topic)[0]
#     test_idxs = np.where(test["Target"] == topic)[0]
#     topic_idxs[topic] = (train_idxs, test_idxs)
# for topic, (train_idxs, test_idxs) in topic_idxs.items():
#     print(topic, len(train_idxs), len(test_idxs))

In [ ]:
pd.crosstab(index=train['Target'], columns=train['Stance'], margins = True)

In [ ]:
pd.crosstab(index=test['Target'], columns=test['Stance'], margins = True)

### SVM on favor vs against classification

In [ ]:
from sklearn.metrics import make_scorer
from commons.functions import *
from submodular.models2 import *
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

lst = ['AGAINST', 'FAVOR']

HYPERPARAMS_GRID1 = {
    "svc__C": [0.1, 1.0, 10.0, 100.0],
    "exp__gamma": [0.0625, 0.125, 0.25, 0.5, 1.0, 2.0],
}
HYPERPARAMS_GRID2 = {
    "svc__C": [0.1, 1.0, 10.0, 100.0]
}

CV = 3
N_JOBS = 8
scorer = make_scorer(bacc)

In [ ]:
ss = set(train.Target.values)
for target in ss:
    train_target = train.query('Target == @target & Stance == @lst').reset_index()
    test_target = test.query('Target == @target & Stance == @lst').reset_index()
    del train_target['index']
    del test_target['index']
    y_tr = np.ones(len(train_target), dtype = np.int8)
    y_te = np.ones(len(test_target), dtype = np.int8)
    y_tr[np.where(train_target.Stance.values == "AGAINST")] = -1
    y_te[np.where(test_target.Stance.values == "AGAINST")] = -1
    train_target["y"] = y_tr
    test_target["y"] = y_te
    print("fitting for " + target)
    Steps = [( [
            ("tok", PandasSelector("tokens")),
            ("idf", BoW(weights = "idf")),
            ("exp", ExpK()),
            ("ung", UnGramMatrix()),
            ("svc", SVC(kernel = "precomputed", class_weight = "balanced")),
        ], HYPERPARAMS_GRID1 ), ( [
            ("tok", PandasSelector("tokens")),
            ("idf", BoW(weights = "idf")),
            ("cos", CosineK()),
            ("ung", UnGramMatrix()),
            ("svc", SVC(kernel = "precomputed", class_weight = "balanced")),
        ], HYPERPARAMS_GRID2), ( [
            ("tok", PandasSelector("tokens")),
            ("idf", BoW(weights = "idf")),
            ("lin", LinearK()),
            ("ung", UnGramMatrix()),
            ("svc", SVC(kernel = "precomputed", class_weight = "balanced")),
        ], HYPERPARAMS_GRID2) 
    ]
    for steps, hyps in Steps:
        clf = GridSearchCV(Pipeline(steps), 
                    hyps, 
                    cv = CV, 
                    n_jobs = N_JOBS, 
                    pre_dispatch = N_JOBS,
                    scoring = scorer, 
                    return_train_score = True, 
                    error_score = 0.5,
                    verbose = 0
        )
        clf.fit(train_target, train_target.y.values)
        train_score, test_score = clf.score(train_target, train_target.y.values), clf.score(test_target, test_target.y.values)
        val_score = clf.best_score_
        # logger.info(clf.cv_results_['mean_test_score'])
        CC , gammaa = clf.best_estimator_.steps[-1][1].C, ""
        try:
            gammaa = clf.best_estimator_.steps[2][1].gamma
        except: 
            pass
        print("{}[{}]: val:{:.4g}, test:{:.4g} |{},{}".format(
            target, steps[2][0], val_score, test_score, CC, gammaa))
    print("\n")

### Comparative summaries evaluations

In [11]:
import pandas as pd
res = pd.read_csv("../semeval2016.6/res_f1.csv")
def split_name(name):
    splitted = name.split("_")
    feats = splitted[1]
    method = "_".join(splitted[2:])
    return feats, method
res["feats"], res["method"] = zip(*[split_name(v) for v in res["method"]])
res.head()

,target,method,k,val,test,L1,L2,f1.Against,f1.For,feats
0,Atheism,exp_greedy-diff,4,0.6552,0.6417,7.8,10.0,0.8339,0.4494,idf-bow
1,Atheism,exp_greedy-diff,8,0.6887,0.6069,8.6,10.0,0.8014,0.4124,idf-bow
2,Atheism,exp_greedy-diff,16,0.6920,0.6239,7.6,10.0,0.8590,0.3889,idf-bow
3,Atheism,exp_greedy-diff,24,0.7121,0.6704,8.4,9.4,0.8523,0.4884,idf-bow
4,Atheism,exp_greedy-diff,32,0.7115,0.6730,8.7,9.9,0.8944,0.4516,idf-bow


In [15]:
pivoted = pd.pivot_table(res, index = ["target", "method", "feats"], columns = ["k"], values = ["f1.Against", "f1.For"], aggfunc=lambda x: x)
pivoted

f1.Against           \
k                                                                4        8    
target                           method          feats                         
Atheism                          cos_nn          bow        0.55900  0.71970   
                                                 idf-bow    0.51980  0.79720   
                                 cos_pr          bow        0.80820  0.46510   
                                                 idf-bow    0.49770  0.66400   
                                 exp_greedy-diff bow        0.77780  0.80820   
                                                 idf-bow    0.83390  0.80140   
                                 exp_greedy-div  bow        0.77780  0.75090   
                                                 idf-bow    0.83390  0.79860   
                                 length          idf-bow    0.42650  0.57500   
Climate Change is a Real Concern cos_nn          bow        0.18600  0.24240   
                                                 idf-bow    0.16220  0.17500   
                                 cos_pr          bow        0.11760  0.19510   
                                                 idf-bow    0.18560  0.10000   
                                 exp_greedy-diff bow        0.15380  0.12770   
                                                 idf-bow    0.16670  0.11430   
                                 exp_greedy-div  bow        0.15690  0.16670   
                                                 idf-bow    0.18180  0.24000   
                                 length          idf-bow    0.19800  0.21050   
Feminist Movement                cos_nn          bow        0.47390  0.43800   
                                                 idf-bow    0.56630  0.66460   
                                 cos_pr          bow        0.44800  0.53790   
                                                 idf-bow    0.81220  0.44180   
                                 exp_greedy-diff bow        0.65840  0.58180   
                                                 idf-bow    0.75000  0.59360   
                                 exp_greedy-div  bow        0.65840  0.56410   
                                                 idf-bow    0.75000  0.60690   
                                 length          idf-bow    0.68240  0.67070   
Hillary Clinton                  cos_nn          bow        0.66190  0.77640   
                                                 idf-bow    0.48760  0.70310   
                                 cos_pr          bow        0.02299  0.01156   
                                                 idf-bow    0.02299  0.01156   
                                 exp_greedy-diff bow        0.71620  0.71530   
                                                 idf-bow    0.79030  0.75560   
                                 exp_greedy-div  bow        0.73220  0.79250   
                                                 idf-bow    0.80470  0.75400   
                                 length          idf-bow    0.31130  0.65960   
Legalization of Abortion         cos_nn          bow        0.82700  0.80700   
                                                 idf-bow    0.78550  0.64830   
                                 cos_pr          bow        0.50190  0.47330   
                                                 idf-bow    0.51090  0.57140   
                                 exp_greedy-diff bow        0.80910  0.64830   
                                                 idf-bow    0.76330  0.83190   
                                 exp_greedy-div  bow        0.60710  0.81050   
                                                 idf-bow    0.75000  0.74010   
                                 length          idf-bow    0.78340  0.79070   

                                                                           \
k                                                              16      24   
target                           method          feats        

In [16]:
# pivoted.to_csv("pivoted_semeval_f1.csv", index = False)
pivoted.to_excel("pivoted_semeval_f1.xlsx")

In [ ]:
res["length"] = 0.5 * ( res["L1"] + res["L2"] )
pd.pivot_table(res, index = ["target", "feats"], columns = ["method"], values = ["length"], aggfunc=lambda x: x)